# Importação das Bibliotecas que serão utilizadas no Modelo

In [ ]:
!pip install --upgrade tensorflow-addons

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import gdown
import plotly.express as px
import plotly.graph_objects as go
from tensorflow.keras import backend as K
from sklearn.cluster import KMeans

## Download dos arquivos contendo os datasets de consumo desde 2019 a 2024
  - Aqui retiramos a base de 2020, por conta da pandemia do Coronavírus. Foi uma escolha do grupo devido à possibilidade de discrepância nas leituras

In [ ]:
arquivo_destino_base = "dataset_{}.csv"

ids = {
    "consumo_2024": "1-iXT7eaJWQokHf9cyfrB8N5wvkdhgjJW",
    "consumo_2023": "1-WfvkRwaRr85B_Joxcm9xVdpyg5NBAmp",
    "consumo_2022": "1-Uu4Tf4lufJVFeJnYKc5w7OeW66pe1eC",
    "consumo_2021": "1-2PsTLzG4dcY4wM0p7vFfabUuLv950gC",
    "consumo_2020": "1-1pOoa0eJlNJ94BMi7p4PTx5KUS96mhX",
    "consumo_2019": "1-2PsTLzG4dcY4wM0p7vFfabUuLv950gC",
    "CONSUMO_GERAL": "1-IOqfwmh_tTIDHeOer8J-HkGFtwuX67g",
}


dataframes = {}


for key, file_id in ids.items():
    url = f"https://drive.google.com/uc?id={file_id}"
    arquivo_destino = arquivo_destino_base.format(key)

    gdown.download(url, arquivo_destino, quiet=False)

In [ ]:
arquivos_csv = [
    "./dataset_consumo_2024.csv",
    "./dataset_consumo_2023.csv",
    "./dataset_consumo_2022.csv",
    "./dataset_consumo_2021.csv",
    "./dataset_consumo_2019.csv",
]

ALL_COLUMNS_CONSUMO_GERAL = pd.concat([pd.read_csv(arquivo, delimiter=";") for arquivo in arquivos_csv], axis=0)

In [ ]:
consumo_geral = pd.read_csv('/content/dataset_CONSUMO_GERAL.csv')

## Download do dataset com o Target das Fraudes

In [ ]:
file_id_fraudes = "1-MbIlChqQapcxFkoJgpbQIsN9FBLfbX1"
url_fraudes = f"https://drive.google.com/uc?id={file_id_fraudes}"

gdown.download(url_fraudes, quiet=False)

In [ ]:
fraudes = pd.read_csv('/content/fraudes.csv')

### A tabela "ALL_COLUMNS_CONSUMO_GERAL" possui todas as tabelas de consumo e a partir disso decidimos considerar algumas colunas categóricas que podem ajudar a melhorar o desempenho do nosso modelo

In [ ]:
ALL_COLUMNS_CONSUMO_GERAL

### Remoção de colunas indesejadas até o momento

In [ ]:
ALL_COLUMNS_CONSUMO_GERAL = ALL_COLUMNS_CONSUMO_GERAL.drop(columns=['Unnamed: 0', 'EMP_CODIGO', 'COD_GRUPO', 'COD_SETOR_COMERCIAL', 'NUM_QUADRA', 'COD_ROTA_LEITURA', 'SEQ_RESPONSAVEL', 'ECO_RESIDENCIAL', 'ECO_COMERCIAL', 'ECO_INDUSTRIAL', 'ECO_PUBLICA', 'ECO_OUTRAS','LTR_ATUAL', 'LTR_COLETADA', 'DAT_LEITURA', 'DIAS_LEITURA', 'COD_LEITURA_INF_1', 'COD_LEITURA_INF_2', 'COD_LEITURA_INF_3', 'HORA_LEITURA', 'DSC_SIMULTANEA', 'COD_LEITURA_INT','EXCECAO'])

In [ ]:
ALL_COLUMNS_CONSUMO_GERAL.columns

### Nessa seção queriamos validar a tabela de "VOLUME_ESTIMADO_ACUM" para ver se ela poderia agregar dentro do nosso modelo

In [ ]:
ALL_COLUMNS_CONSUMO_GERAL[['VOLUME_ESTIMADO_ACUM']].nunique()

In [ ]:
ALL_COLUMNS_CONSUMO_GERAL[ALL_COLUMNS_CONSUMO_GERAL['VOLUME_ESTIMADO'] != 0]

#### O insight retirado aqui é que talvez a melhor coluna para validação e ser utilizada como featura no modelo é a coluna de Volume Estimado. Ela possui maior consistência nos resultados, do que a coluna de Volume Estimado Acumulado

### Separação de Features relevantes

Após pesquisas e visualizar as colunas disponíveis, percebemos uma coluna que poderia ser interessante para o processo de identificação de fraude. A coluna de "DSC_OCORRENCIA". Ela basicamente corresponde a descrição de como foi o processo de vistoria e coleta do responsável e em cada um dos domicílios (matrícula)

Isso surgiu como uma possibilidade de tentar direcionar o modelo para casos nos quais há uma maior possibilidade de uma fraude, de acordo com a visualização do medidor em cada um desses domicílios.

In [ ]:
ALL_COLUMNS_CONSUMO_GERAL_PREMISSA_VINI = ALL_COLUMNS_CONSUMO_GERAL[ALL_COLUMNS_CONSUMO_GERAL['DSC_OCORRENCIA'].isin([
    'NORMAL',
    'MEDIDOR RETIRADO/FURTADO',
    'LEITURA COLETADA PELO CLIENTE',
    'MEDIDOR NÃO LOCALIZADO',
    'IMÓVEL DESOCUPADO'
])]

ALL_COLUMNS_CONSUMO_GERAL_PREMISSA_VINI

## Tratando Dataframe com mais colunas targets adicionadas

#### Seperação do Dataframe apenas para a visualização das matriculas com Categoria Pessoa Jurídica

In [ ]:
dataframe_pj_premissa = ALL_COLUMNS_CONSUMO_GERAL_PREMISSA_VINI[ALL_COLUMNS_CONSUMO_GERAL_PREMISSA_VINI["CATEGORIA"].isin(["COMERCIAL", "PUBLICA", "INDUSTRIAL"])]
dataframe_pj_premissa

### Processo de One Hot Encoding para as Colunas, as quais serão as features para o nosso modelo

In [ ]:
dataframe_pj_premissa = pd.get_dummies(dataframe_pj_premissa, columns=['TIPO_LIGACAO', 'DSC_OCORRENCIA', 'STA_TROCA', 'STA_ACEITA_LEITURA'], dtype='int')

In [ ]:
dataframe_pj_premissa

In [ ]:
fraudes.columns

In [ ]:
fraudes

In [ ]:
dataframe_fraudes_premissa = fraudes[['MATRICULA', 'DESCRICAO']].drop_duplicates()
dataframe_fraudes_premissa = pd.get_dummies(dataframe_fraudes_premissa, columns=['DESCRICAO'], dtype='int')

In [ ]:
dataframe_fraudes_premissa['DESCRICAO_IRREGULARIDADE IDENTIFICADA'].unique()

In [ ]:
len(dataframe_fraudes_premissa)

In [ ]:
dataframe_pj_premissa = pd.merge(dataframe_pj_premissa, dataframe_fraudes_premissa, on='MATRICULA', how='left')

In [ ]:
# dataframe_pj_premissa.drop_duplicates(subset="MATRICULA", keep='first')
dataframe_pj_premissa.dropna(subset="REFERENCIA")

In [ ]:
dataframe_pj_premissa = dataframe_pj_premissa.dropna(subset="COD_LATITUDE")
dataframe_pj_premissa = dataframe_pj_premissa.dropna(subset="COD_LONGITUDE")

### Agrupamento dos clusteres por Latitude e Longitude

Aqui removendo os outliers nas colunas de latitude e longitude

In [ ]:
def remove_rows_with_column_value_greater_than_one(df, column_names):
    condition = (df[column_names] > -10).any(axis=1)

    df_filtered = df[~condition]

    return df_filtered

dataframe_pj_premissa = remove_rows_with_column_value_greater_than_one(dataframe_pj_premissa, ['COD_LATITUDE', 'COD_LONGITUDE'])

In [ ]:
def plotar(n_clusters):

  df_temp = dataframe_pj_premissa.copy()
  kmeans = KMeans(n_clusters=n_clusters, random_state=42)
  df_temp['cluster'] = kmeans.fit_predict(dataframe_pj_premissa[['COD_LATITUDE', 'COD_LONGITUDE']])

  # visualizando
  plt.figure(figsize=(10, 6))
  plt.scatter(dataframe_pj_premissa['COD_LONGITUDE'], dataframe_pj_premissa['COD_LATITUDE'], c=df_temp['cluster'], cmap='viridis', marker='o', s=100)
  plt.title(f"K-Means Clustering com {n_clusters} Clusters")
  plt.xlabel("Longitude")
  plt.ylabel("Latitude")
  plt.show()



# visulizar clusters do range de 2 para 17
for x in range(2, 17):
  print()
  plotar(x)

In [ ]:
kmeans = KMeans(n_clusters=20, random_state=42)
dataframe_pj_premissa['cluster'] = kmeans.fit_predict(dataframe_pj_premissa[['COD_LATITUDE', 'COD_LONGITUDE']])

In [ ]:
dataframe_pj_premissa = dataframe_pj_premissa.drop(columns=['COD_LATITUDE', 'COD_LONGITUDE'])

In [ ]:
dataframe_pj_premissa = pd.get_dummies(dataframe_pj_premissa, columns = ['cluster'], dtype=int)

### Normalizando com o Robust Scaler

O RobustScaler é uma técnica de normalização usada para transformar dados. Ele é útil especialmente quando os dados contêm outliers, ou seja, valores atípicos que podem distorcer (neste caso o Consumo e o Volume) os resultados de outras técnicas de escalonamento, como StandardScaler ou MinMaxScaler.

O RobustScaler transforma os dados subtraindo a mediana e dividindo pela amplitude interquartil (IQR, Interquartile Range). A mediana é o valor do ponto médio quando os dados são ordenados, e o IQR é a diferença entre o terceiro quartil (75º percentil) e o primeiro quartil (25º percentil).

Esse método é menos sensível a outliers porque, ao contrário da média e do desvio padrão (usados pelo StandardScaler), a mediana e o IQR não são afetados por valores extremos.

In [ ]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()

dataframe_pj_premissa[['CONS_MEDIDO']] = scaler.fit_transform(dataframe_pj_premissa[['CONS_MEDIDO']])
dataframe_pj_premissa[['VOLUME_ESTIMADO']] = scaler.fit_transform(dataframe_pj_premissa[['VOLUME_ESTIMADO']])

In [ ]:
pivoted_df = pd.pivot_table(
    dataframe_pj_premissa,
    index='MATRICULA',
    columns='REFERENCIA',
    values=['CONS_MEDIDO', 'VOLUME_ESTIMADO'],
    aggfunc='sum'
)

pivoted_df.columns = ['_'.join(col).strip() for col in pivoted_df.columns.values]
pivoted_df = pivoted_df.reset_index()

pivoted_df.head()

In [ ]:
len(pivoted_df)

### Atribuição de cada uma das variáveis categóricas ao dataframe com o consumo e volume históricos

- É importante citar aqui que atribuímos a premissa que a primeira definição de tipo de ligação, descrição e os outros utilizados, serão os que tomaremos como base para inferência dentro do modelo

In [ ]:
tipo_ligacao = dataframe_pj_premissa[['MATRICULA','TIPO_LIGACAO_Consumo Fixo', 'TIPO_LIGACAO_Hidrometrado']].drop_duplicates(subset='MATRICULA', keep='first')
pivoted_df = pivoted_df.merge(tipo_ligacao, on='MATRICULA', how='left').fillna(0)

In [ ]:
descricao_ocorrencia = dataframe_pj_premissa[['MATRICULA','DSC_OCORRENCIA_MEDIDOR NÃO LOCALIZADO', 'DSC_OCORRENCIA_MEDIDOR RETIRADO/FURTADO', 'DSC_OCORRENCIA_NORMAL']].drop_duplicates(subset='MATRICULA', keep='first')
pivoted_df = pivoted_df.merge(descricao_ocorrencia, on='MATRICULA', how='left').fillna(0)

In [ ]:
fraude_ou_não = dataframe_pj_premissa[['MATRICULA','DESCRICAO_IRREGULARIDADE IDENTIFICADA']].drop_duplicates(subset='MATRICULA', keep='first')
pivoted_df = pivoted_df.merge(fraude_ou_não, on='MATRICULA', how='left').fillna(0)

In [ ]:
dataframe_pj_premissa.columns.values

In [ ]:
clusters = dataframe_pj_premissa[['MATRICULA','cluster_0', 'cluster_1', 'cluster_2', 'cluster_3', 'cluster_4',
       'cluster_5', 'cluster_6', 'cluster_7', 'cluster_8', 'cluster_9',
       'cluster_10', 'cluster_11', 'cluster_12', 'cluster_13',
       'cluster_14', 'cluster_15', 'cluster_16', 'cluster_17',
       'cluster_18', 'cluster_19']].drop_duplicates(subset='MATRICULA', keep='first')
pivoted_df = pivoted_df.merge(clusters, on='MATRICULA', how='left').fillna(0)

In [ ]:
pivoted_df = pivoted_df.fillna(0)

In [ ]:
pivoted_df

In [ ]:
colunas_data = ['CONS_MEDIDO_2019-01-01', 'CONS_MEDIDO_2019-02-01',
       'CONS_MEDIDO_2019-03-01', 'CONS_MEDIDO_2019-04-01',
       'CONS_MEDIDO_2019-05-01', 'CONS_MEDIDO_2019-06-01',
       'CONS_MEDIDO_2019-07-01', 'CONS_MEDIDO_2019-08-01',
       'CONS_MEDIDO_2019-09-01', 'CONS_MEDIDO_2019-10-01',
       'CONS_MEDIDO_2019-11-01', 'CONS_MEDIDO_2019-12-01',
       'CONS_MEDIDO_2022-01-01', 'CONS_MEDIDO_2022-02-01',
       'CONS_MEDIDO_2022-03-01', 'CONS_MEDIDO_2022-04-01',
       'CONS_MEDIDO_2022-05-01', 'CONS_MEDIDO_2022-06-01',
       'CONS_MEDIDO_2022-07-01', 'CONS_MEDIDO_2022-08-01',
       'CONS_MEDIDO_2022-09-01', 'CONS_MEDIDO_2022-10-01',
       'CONS_MEDIDO_2022-11-01', 'CONS_MEDIDO_2022-12-01',
       'CONS_MEDIDO_2023-01-01', 'CONS_MEDIDO_2023-02-01',
       'CONS_MEDIDO_2023-03-01', 'CONS_MEDIDO_2023-04-01',
       'CONS_MEDIDO_2023-05-01', 'CONS_MEDIDO_2023-06-01',
       'CONS_MEDIDO_2023-07-01', 'CONS_MEDIDO_2023-08-01',
       'CONS_MEDIDO_2023-09-01', 'CONS_MEDIDO_2023-10-01',
       'CONS_MEDIDO_2023-11-01', 'CONS_MEDIDO_2023-12-01',
       'CONS_MEDIDO_2024-01-01', 'CONS_MEDIDO_2024-02-01',
       'CONS_MEDIDO_2024-03-01', 'CONS_MEDIDO_2024-04-01',
       'CONS_MEDIDO_2024-05-01', 'CONS_MEDIDO_2024-06-01',
       'CONS_MEDIDO_2024-07-01', 'CONS_MEDIDO_2024-08-01']

In [ ]:
np.unique(pivoted_df['DESCRICAO_IRREGULARIDADE IDENTIFICADA'].values, return_counts=True)

## Rodando o modelo

### Divisão de Treino e Teste

In [ ]:
pivoted_df.shape

#### Balanceando os dados com Smote (Synthetic Minority Over-sampling Technique)

O smote é uma técnica de "oversampling" usada para balancear datasets com o objetivo de aumentar a quantidade de amostras da classe minoritária gerando novos exemplos sintéticos, em vez de simplesmente replicar os dados existentes. Isso ajuda a melhorar a performance de modelos de machine learning ao treinar com um dataset mais balanceado.

Quando você treina um modelo de machine learning com um dataset desbalanceado (onde uma classe tem muito mais exemplos do que a outra), o modelo tende a favorecer a classe majoritária.

Como resultado, o modelo pode ter um bom desempenho em termos de acurácia geral, mas um desempenho ruim ao identificar a classe minoritária (por exemplo, falhas, fraudes, etc.).

O Smote ajuda a mitigar esse problema ao balancear o dataset, aumentando o número de exemplos da classe minoritária.

In [ ]:
def balanciar(df):
  smote = SMOTE(random_state=42)
  X = pivoted_df.drop('DESCRICAO_IRREGULARIDADE IDENTIFICADA', axis=1)
  y = pivoted_df['DESCRICAO_IRREGULARIDADE IDENTIFICADA']
  X_res, y_res = smote.fit_resample(X, y)
  return pd.concat([X_res, y_res], axis=1)

In [ ]:
# from imblearn.under_sampling import RandomUnderSampler

# def balanciar(df):
#   rus = RandomUnderSampler(random_state=42)
#   X = pivoted_df.drop('DESCRICAO_IRREGULARIDADE IDENTIFICADA', axis=1)
#   y = pivoted_df['DESCRICAO_IRREGULARIDADE IDENTIFICADA']
#   X_res, y_res = rus.fit_resample(X, y)
#   return pd.concat([X_res, y_res], axis=1)

In [ ]:
# pivoted_df = balanciar(pivoted_df)

In [ ]:
np.unique(pivoted_df['DESCRICAO_IRREGULARIDADE IDENTIFICADA'].values, return_counts=True)

In [ ]:
pivoted_df = balanciar(pivoted_df.fillna(0))

In [ ]:
pivoted_df.columns

In [ ]:
y = pivoted_df['DESCRICAO_IRREGULARIDADE IDENTIFICADA'].values
X = pivoted_df.drop(['MATRICULA','DESCRICAO_IRREGULARIDADE IDENTIFICADA'], axis=1).values

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
import tensorflow.keras as keras
from tensorflow.keras import layers

model = keras.Sequential([
    keras.layers.Input(shape=(x_train.shape[1],)),
    keras.layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    keras.layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    keras.layers.Dense(32, activation='relu'),
    layers.Dropout(0.2),
    keras.layers.Dense(16, activation='relu'),
    layers.Dropout(0.2),
    keras.layers.Dense(8, activation='relu'),
    layers.Dropout(0.2),
    keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'precision', 'recall'])

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
x_val.shape

In [ ]:
y_val.shape

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

result = model.fit(x_train, y_train, epochs=45, batch_size=64, validation_data=(x_val, y_val))

In [ ]:
history = result.history
fig = px.line(x=list(range(1, 46)), y=history['loss'], labels={'x': 'Épocas', 'y': 'Perda'}, title='Função de Custo durante o Treinamento')
fig.update_traces(mode='lines+markers')
fig.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from keras.models import load_model

y_pred = model.predict(x_val)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_val, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Não é fraude", "É fraude"])
disp.plot(cmap=plt.cm.Blues)
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
plt.plot(result.history['loss'], label='loss')
plt.plot(result.history['val_loss'], label='val_loss')
plt.legend(['Erro Treino', 'Erro Teste'])
plt.xlabel('Épocas')
plt.ylabel('Função de Custo')
plt.title('Training Validation')
plt.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Line(x=list(range(1, 26)), y=history['accuracy'], mode='lines+markers', name='Precisão de Treinamento'))

fig.add_trace(go.Line(x=list(range(1, 26)), y=history['val_loss'], mode='lines+markers', name='Perda de Validação'))

fig.update_layout(title='Precisão e Perda durante o Treinamento', xaxis_title='Épocas', yaxis_title='Valor', legend_title='Métrica')
fig.show()

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model, to_file='model_visualization.png', show_shapes=True, show_layer_names=True)

### Agregação de hiperparâmetros

In [ ]:
%pip install scikit-optimize

In [ ]:
from skopt import gp_minimize
from skopt.space import Real, Integer
from sklearn.model_selection import train_test_split
from tensorflow import keras

# Definindo a função objetivo
def objective(params):
    learning_rate, num_neurons, batch_size, dropout_rate = params

    # Pegando a última 'val_loss' como métrica para otimização
    val_loss = result.history['val_loss'][-1]
    return val_loss

# Definindo o espaço de busca
space = [
    Real(1e-4, 1e-2, "log-uniform", name='learning_rate'),  # Taxa de aprendizado
    Integer(16, 256, name='num_neurons'),                 # Número de neurônios
    Integer(16, 128, name='batch_size'),                   # Tamanho do batch
    Real(0.1, 0.5, name='dropout_rate')                   # Olhar pro dropout
]

# Otimizando usando a otimização bayesiana
result = gp_minimize(objective, space, n_calls=30, random_state=42)

print(f"Melhores hiperparâmetros: {result.x}")
print(f"Menor val_loss: {result.fun}")

In [ ]:
print(f"Melhor taxa de aprendizado: {result.x[0]}, Melhor número de neurônios: {result.x[1]}")

### Random Search

In [ ]:
def create_model(learning_rate=0.001, dropout_rate=0.2):
    model = keras.Sequential([
        keras.layers.Input(shape=(x_train.shape[1],)),
        keras.layers.Dense(128, activation='relu'),
        layers.Dropout(dropout_rate),
        keras.layers.Dense(64, activation='relu'),
        layers.Dropout(dropout_rate),
        keras.layers.Dense(32, activation='relu'),
        layers.Dropout(dropout_rate),
        keras.layers.Dense(16, activation='relu'),
        layers.Dropout(dropout_rate),
        keras.layers.Dense(8, activation='relu'),
        layers.Dropout(dropout_rate),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
%pip install scikeras

In [ ]:
from sklearn.metrics import fbeta_score, make_scorer, accuracy_score, precision_score, recall_score
from sklearn.model_selection import RandomizedSearchCV
from scikeras.wrappers import KerasClassifier


model = KerasClassifier(model=create_model, optimizer='adam')

param_dist = {
    'batch_size': [64, 128],
    'epochs': [30, 45],
}

# Define the scoring metrics
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score)}

random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=10,
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=1,
    scoring=scoring,
    refit='accuracy'
)

random_search.fit(x_train, y_train)

print(random_search.best_params_)
best_model = random_search.best_estimator_